In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# loading training data    
data = np.load('Data/species/species_train.npz')
train_locs = data['train_locs']  # 2D array, rows are number of datapoints and 
                                 # columns are "latitude" and "longitude"
train_ids = data['train_ids']    # 1D array, entries are the ID of the species 
                                 # that is present at the corresponding location in train_locs
species = data['taxon_ids']      # list of species IDe. Note these do not necessarily start at 0 (or 1)
species_names = dict(zip(data['taxon_ids'], data['taxon_names']))  # latin names of species 

# loading test data 
data_test = np.load('Data/species/species_test.npz', allow_pickle=True)
test_locs = data_test['test_locs']    # 2D array, rows are number of datapoints 
                                     # and columns are "latitude" and "longitude"
# data_test['test_pos_inds'] is a list of lists, where each list corresponds to 
# the indices in test_locs where a given species is present, it can be assumed 
# that they are not present in the other locations 
test_pos_inds = dict(zip(data_test['taxon_ids'], data_test['test_pos_inds']))    

In [31]:
test_ids = data_test['taxon_ids']

In [29]:
testdata_test['taxon_ids']

500

In [9]:
ids= pd.DataFrame(train_ids)
locs= pd.DataFrame(train_locs)

In [17]:
data_train = pd.concat([ids,locs],axis=1)
data_train.columns = ['id', 'lat', 'long']
data_train

,id,lat,long
0,31529,-18.286728,143.481247
1,31529,-13.099798,130.783646
2,31529,-13.965274,131.695145
3,31529,-12.853950,132.800507
4,31529,-12.196790,134.279327
...,...,...,...
272032,145031,33.716885,73.203621
272033,145031,24.600239,72.730560
272034,145031,18.849600,80.654129
272035,145031,21.073837,75.945656


# Linear Regression

In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = data_train[['id']]
Y = data_train[['lat','long']]

model = LinearRegression()

model.fit(X, Y)

Y_pred = model.predict(X)

mse = mean_squared_error(Y,Y_pred)
mse


4772.316999823453

# Mean Value

In [49]:
grouped_data = data_train.groupby('id').agg({
    'lat': 'mean',  # or 'first', 'last', etc.
    'long': 'mean'})

In [50]:
grouped_data

,lat,long
id,,
458,-23.635391,-46.610573
460,-31.017895,-57.912819
487,-25.073004,20.192400
585,33.882202,80.526962
871,35.205753,-2.006270
...,...,...
1289606,56.475147,-75.294388
1289647,-22.456312,-49.856724
1289681,22.039534,-79.168694


# K-Nearest Neighbors

In [40]:
data_train1 = data_train[:10000]


In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error


# Assuming 'df' is your DataFrame with 'species_id', 'plan', and 'lot'
# Step 1: Create dummy variables for species_id
dummies = pd.get_dummies(data_train1['id'], prefix='species')
df_with_dummies = pd.concat([data_train1.drop('id', axis=1), dummies], axis=1)

# Step 2: Prepare features and target
X = df_with_dummies.drop(columns=['lat', 'long'])  # Features (dummy variables)
y = df_with_dummies[['lat', 'long']]                # Target (plan and lot)


# Step 5: Set up KNN Regressor with MultiOutputRegressor for multiple targets
knn = KNeighborsRegressor()
multi_output_knn = MultiOutputRegressor(knn)

# Step 6: Use GridSearchCV to optimize the number of neighbors 'k'
param_grid = {'estimator__n_neighbors': list(range(1, 30))}  # Testing k from 1 to 30
grid_search = GridSearchCV(multi_output_knn, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)

# Get the best model
best_knn = grid_search.best_estimator_
best_k = grid_search.best_params_['estimator__n_neighbors']
print(f"Optimal number of neighbors: {best_k}")


Optimal number of neighbors: 3


NameError: name 'X_test_scaled' is not defined

3117

In [47]:
dummies_test = pd.get_dummies(test_ids[:100], prefix='species')
y_pred = best_knn.predict(dummies_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- species_10090
- species_1078
- species_117054
- species_12716
- species_12735
- ...
